In [1]:
#pyright: reportPrivateImportUsage=false, reportGeneralTypeIssues=false
import functools
import logging
import os
import subprocess
import warnings

# import hydra
# import hydra.core.hydra_config
# import omegaconf

# from analytic_svd_model import AnalyticSVDModel

# #Register additional resolver for log path
# omegaconf.OmegaConf.register_new_resolver("list_to_string", lambda o: functools.reduce(lambda acc, x: acc+", "+x.replace("\"","").replace("/"," "), o, "")[2:])
# omegaconf.OmegaConf.register_new_resolver("eval", lambda c: eval(c))

import pytorch_lightning
import pytorch_lightning.accelerators
import pytorch_lightning.callbacks
import pytorch_lightning.loggers
import pytorch_lightning.utilities

import torch
import torch.utils.tensorboard
import torch.version

# from mnist_datamodule import MNISTDataModule
from ellipses_datamodule import EllipsesDataModule
# from learned_filter_model import LearnedFilterModel
# from analytic_filter_model import AnalyticFilterModel
# from learned_svd_model import LearnedSVDModel
# from analytic_svd_model import AnalyticSVDModel

import typing
import sys
check_python_version = (sys.version_info[0] >= 3) and (sys.version_info[1] >= 10)

In [2]:
config = {'model': {
            'name': 'learned', 
            'initialization': 'zeros'}, 
        'dataset': {
            'name': 'ellipses', 
            'img_size': 256, 
            'ellipse_count': 10, 
            'ellipse_size': 50, 
            'ellipse_size_min': 10, 
            'blurred': False}, 
        'out_dir': './LearnedRadonFilters', 
        'deterministic': True, 
        'seed': 1234567, 
        'device': 'cuda', 
        'num_workers': 4, 
        'checkpoint': None, 
        'validation_split_percent': 20, 
        'training_batch_size': 32, 
        'shuffle_training_data': True, 
        'drop_last_training_batch': False, 
        'validation_batch_size': 100, #'${training_batch_size}', 
        'shuffle_validation_data': False, 
        'drop_last_validation_batch': False, 
        'test_batch_size': 100, #'${training_batch_size}', 
        'shuffle_test_data': False, 
        'drop_last_test_batch': True, 
        'sino_angles': None, 
        'sino_positions': None, 
        'noise_level': 0.004, 
        'optimizer_lr': 0.05, 
        'epochs': 1, 
        'training_batch_count': -1, 
        'validation_batch_count': -1, 
        'test_batch_count': -1}

In [3]:
import pickle
with open('config.pkl', 'rb') as fp:
    config_v2 = pickle.load(fp)

In [4]:
import pickle
with open('config_09nov.pkl', 'rb') as fp:
    config_v3 = pickle.load(fp)

In [4]:
print(type(config))
print(type(config_v2))

<class 'dict'>
<class 'omegaconf.dictconfig.DictConfig'>


In [5]:
print(config)

{'model': {'name': 'learned', 'initialization': 'zeros'}, 'dataset': {'name': 'ellipses', 'img_size': 256, 'ellipse_count': 10, 'ellipse_size': 50, 'ellipse_size_min': 10, 'blurred': False}, 'out_dir': './LearnedRadonFilters', 'deterministic': True, 'seed': 1234567, 'device': 'cuda', 'num_workers': 4, 'checkpoint': None, 'validation_split_percent': 20, 'training_batch_size': 32, 'shuffle_training_data': True, 'drop_last_training_batch': False, 'validation_batch_size': 100, 'shuffle_validation_data': False, 'drop_last_validation_batch': False, 'test_batch_size': 100, 'shuffle_test_data': False, 'drop_last_test_batch': True, 'sino_angles': None, 'sino_positions': None, 'noise_level': 0.004, 'optimizer_lr': 0.05, 'epochs': 1, 'training_batch_count': -1, 'validation_batch_count': -1, 'test_batch_count': -1}


In [6]:
#(200 if self.config.test_batch_count == -1 else self.config.test_batch_count)*self.config.test_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, test_transform if self.config.dataset.blurred else None

In [7]:
print(config_v2.test_batch_count)
print(config_v2.test_batch_size )

print(config_v2.dataset.img_size )
print(config_v2.dataset.ellipse_count)
print(config_v2.dataset.ellipse_size)
print(config_v2.dataset.ellipse_size_min)

print(config_v2.dataset.blurred)

-1
32
256
10
50
10
False


In [8]:
#config_v2.test_batch_count = 1
#config_v2.test_batch_size  = 1000

config_v2.dataset.img_size  = 256 # 64 #256
config_v2.dataset.ellipse_count = 1
config_v2.dataset.ellipse_size = 40
config_v2.dataset.ellipse_size_min = 40

config_v2.dataset.blurred = False

In [9]:
config_v2_reduced_model = config_v2.copy()
config_v2_reduced_model.dataset.img_size = 64
print(config_v2_reduced_model.dataset.img_size)
print(config_v2.dataset.img_size)

64
256


In [10]:
datamodule = EllipsesDataModule(config_v2)

In [11]:
all_images = []
cont = 0
for X in datamodule.test_dataloader():
    cont += 1
    #all_images.append(X[0])

print(cont)

200


In [12]:
#cont = 0
#for X in iter(datamodule.test_dataloader()):
#    cont += 1

#print(cont)

In [12]:
X[0].shape

torch.Size([32, 1, 256, 256])

In [12]:
X[0].shape

torch.Size([1000, 1, 64, 64])

In [13]:
X[0].shape

torch.Size([1000, 1, 64, 64])

In [14]:
X[0][0].shape[-2:]

torch.Size([64, 64])

In [5]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

In [14]:
plt.imshow(X[0][5].view(X[0][0].shape[-2:]))
plt.show()

In [37]:
my_img = X[0][7]
type(my_img)

torch.Tensor

In [6]:
import torchvision

In [16]:
v_compose = []
v_compose.append(torchvision.transforms.Resize(size=64))#config.resized_shape))

if len(v_compose)==0:
    dataset_transform = None
else:
    dataset_transform = torchvision.transforms.Compose(v_compose)

In [17]:
# Resize funziona su un batch? --> Risposta: sì
print(X[0].shape)
print(dataset_transform)
print(dataset_transform(X[0]).shape)

fig, axs = plt.subplots(2)
fig.suptitle('Test Ellipses')
axs[0].imshow(X[0][2].view(X[0][2].shape[-2:]))
axs[1].imshow(dataset_transform(X[0])[2].view((dataset_transform(X[0])[2]).shape[-2:]))
plt.show()

torch.Size([32, 1, 256, 256])
Compose(
    Resize(size=64, interpolation=bilinear, max_size=None, antialias=None)
)
torch.Size([32, 1, 64, 64])


In [19]:
v_compose = []
v_compose.append(torchvision.transforms.Resize(size=256))#config.resized_shape))

if len(v_compose)==0:
    dataset_transform = None
else:
    dataset_transform = torchvision.transforms.Compose(v_compose)

In [20]:
# Seguito del precedente: ho ottenuto la stessa identica immagine? --> Risposta: sì
print(all((X[0] == dataset_transform(X[0])).tolist()))

True


In [18]:
v_compose = []
v_compose.append(torchvision.transforms.Resize(size=64))#config.resized_shape))

if len(v_compose)==0:
    dataset_transform = None
else:
    dataset_transform = torchvision.transforms.Compose(v_compose)

In [19]:
# Che fa il modello? Lo verifico sui dati

# 1) Carica il modello
from learned_filter_model import LearnedFilterModel
modelClass = LearnedFilterModel
model = modelClass(config_v2)
reduced_model = modelClass(config_v2_reduced_model)

# 2) Usa i dati vecchi sul modello
print(X[0].shape)
output_model = model.test_step(batch=X, batch_idx=0)
#print(output_model.shape)


# 3) Usa i dati resized sul modello
print(dataset_transform(X[0]).shape)
output_model_resized = reduced_model.test_step(batch=[dataset_transform(X[0]), torch.arange(0,32,1)], batch_idx=0)
#print(output_model_resized.shape)


# 4) plotta i risultati (prossimi 2 blocchi)

torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 64, 64])


In [20]:
print(len(output_model_resized))
print(type(output_model_resized))
print(output_model_resized.keys())
print(output_model_resized['sinogram'].shape)
print(output_model_resized['noisy_sinogram'].shape)
print(output_model_resized['ground_truth'].shape)
print(output_model_resized['reconstruction'].shape)



4
<class 'dict'>
dict_keys(['sinogram', 'noisy_sinogram', 'ground_truth', 'reconstruction'])
torch.Size([32, 1, 256, 93])
torch.Size([32, 1, 256, 93])
torch.Size([32, 1, 64, 64])
torch.Size([32, 1, 64, 64])


In [21]:
print(len(output_model))
print(type(output_model))
print(output_model.keys())
print(output_model['sinogram'].shape)
print(output_model['noisy_sinogram'].shape)
print(output_model['ground_truth'].shape)
print(output_model['reconstruction'].shape)


matplotlib.use('TkAgg')

img_idx = 6

fig, axs = plt.subplots(2, 4)
fig.suptitle('Test reconstruction on original data vs resized data')
axs[0,0].imshow(output_model['sinogram'][img_idx].view((output_model['sinogram'][img_idx]).shape[-2:]))
axs[0,1].imshow(output_model['noisy_sinogram'][img_idx].view((output_model['noisy_sinogram'][img_idx]).shape[-2:]))
axs[1,0].imshow(output_model['ground_truth'][img_idx].view((output_model['ground_truth'][img_idx]).shape[-2:]))
axs[1,1].imshow(output_model['reconstruction'][img_idx].detach().view((output_model['reconstruction'][img_idx]).shape[-2:]).numpy())# lui viene vuoto. Ci sta, il modello non ha ancora imparato nulla

axs[0,2].imshow(output_model_resized['sinogram'][img_idx].view((output_model_resized['sinogram'][img_idx]).shape[-2:]))
axs[0,3].imshow(output_model_resized['noisy_sinogram'][img_idx].view((output_model_resized['noisy_sinogram'][img_idx]).shape[-2:]))
axs[1,2].imshow(output_model_resized['ground_truth'][img_idx].view((output_model_resized['ground_truth'][img_idx]).shape[-2:]))
axs[1,3].imshow(output_model_resized['reconstruction'][img_idx].detach().view((output_model_resized['reconstruction'][img_idx]).shape[-2:]).numpy()) # lui viene vuoto. Ci sta, il modello non ha ancora imparato nulla
plt.show()

4
<class 'dict'>
dict_keys(['sinogram', 'noisy_sinogram', 'ground_truth', 'reconstruction'])
torch.Size([32, 1, 256, 365])
torch.Size([32, 1, 256, 365])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])


In [26]:
#fig, axs = plt.subplots(2, 2)
#fig.suptitle('Test reconstruction on original data')
#axs[0,0].imshow(output_model['sinogram'][img_idx].view((output_model['sinogram'][img_idx]).shape[-2:]))
#axs[0,1].imshow(output_model['noisy_sinogram'][img_idx].view((output_model['noisy_sinogram'][img_idx]).shape[-2:]))
#axs[1,0].imshow(output_model['ground_truth'][img_idx].view((output_model['ground_truth'][img_idx]).shape[-2:]))
#axs[1,1].imshow(output_model['reconstruction'][img_idx].detach().view((output_model['reconstruction'][img_idx]).shape[-2:]).numpy())
#plt.show()

In [7]:
config_v3.validation_batch_size

32

In [8]:
config_v3.resized_shape = 128 #512 #128

In [9]:
from ellipses_datamodule import EllipsesDataModule

In [10]:
datamodule = EllipsesDataModule(config_v3)

In [11]:
datamodule.dataset_transform

Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=None)
)

In [12]:
datamodule.test_dataloader().dataset.transform

channel_selection_mode: noalpha


Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=None)
)

In [60]:
X = next(iter(datamodule.test_dataloader()))


In [17]:
X[0].shape

torch.Size([32, 1, 128, 128])

In [13]:
cont = 0
for Y in datamodule.test_dataloader():
    cont += 1
    break
    #all_images.append(X[0])

print(cont)

channel_selection_mode: noalpha
1


train (batches, images per batch, total)       640, 32, 20480
<br>test        200     32 =  6400
<br>validation  160     32 =  5120

In [55]:
Y[0].shape

torch.Size([32, 1, 128, 128])

In [38]:
(datamodule.test_dataloader().dataset.transform(Y[0])).shape

torch.Size([32, 1, 256, 256])

In [39]:
v_compose = []
v_compose.append(torchvision.transforms.Resize(size=64)) # !!! IMPORTANT !!! comment this line if you want "original"

dataset_transform_postprocessing = torchvision.transforms.Compose(v_compose)

In [40]:
(dataset_transform_postprocessing(Y[0])).shape

torch.Size([32, 1, 64, 64])

In [41]:
fig, axs = plt.subplots(2)
fig.suptitle('Test Ellipses')
axs[0].imshow(Y[0][2].view(Y[0][2].shape[-2:]))
axs[1].imshow((dataset_transform_postprocessing(Y[0][2])).view((dataset_transform_postprocessing(Y[0][2])).shape[-2:]))
plt.show()

In [14]:
plt.imshow(Y[0][6].view(Y[0][6].shape[-2:]))
plt.show()

In [15]:
print(torch.histogram(Y[0][0]))

torch.return_types.histogram(
hist=tensor([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0., 16384.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.]),
bin_edges=tensor([0.5000, 0.5100, 0.5200, 0.5300, 0.5400, 0.5500, 0

In [33]:
import omegaconf
test_config = omegaconf.OmegaConf.create(config)


In [35]:
test_config.test_batch_count

-1

In [2]:
import omegaconf

In [16]:
my_config = {'color':{'name': 'blue', 'fav': True}, 'give_number':5} 
my_omega_config = omegaconf.OmegaConf.create(my_config) 

In [17]:
class test_omega_config():
    def __init__(self, config: omegaconf.DictConfig) -> None:
        self.config = config.copy()

    def print_number(self):
        print(self.config.give_number)

In [18]:
my_class = test_omega_config(my_omega_config)

In [19]:
print(my_omega_config.give_number)
my_class.print_number()

my_omega_config.give_number = 10

print(my_omega_config.give_number)
my_class.print_number()


5
5
10
5


In [13]:
A = my_omega_config.copy()

In [15]:
A.give_number

10

Now I want to check whether the dataloader is deterministic

In [20]:
from ellipses_datamodule import EllipsesDataModule

In [3]:
import pickle
with open('config_09nov.pkl', 'rb') as fp:
    config_v3 = pickle.load(fp)

In [22]:
datamodule = EllipsesDataModule(config_v3)

In [16]:
#pyright: reportGeneralTypeIssues=false

from math import atan, cos, sin, tan
import typing

import omegaconf
import pytorch_lightning as pl
import torch
import torch.utils.data
import torchvision.transforms
import torchvision.transforms.functional
import matplotlib
matplotlib.use("agg")
import matplotlib.patches
import matplotlib.pyplot as plt
import numpy as np

import sys
check_python_version = (sys.version_info[0] >= 3) and (sys.version_info[1] >= 10)


class _my_EllipsesDataset(torch.utils.data.Dataset):
    def __init__(self, img_count: int, img_size: int, ellipses_count: int, ellipses_size: float, ellipses_size_min: float=1, transform: typing.Callable[[torch.Tensor],torch.Tensor]|None if check_python_version else typing.Union[typing.Callable[[torch.Tensor],torch.Tensor] , None] = None, number_ellipses_poisson=False, random_size=True, generator: torch.Generator|None if check_python_version else typing.Union[torch.Generator, None] =None):
        self.img_count = img_count
        self.img_size = img_size
        if number_ellipses_poisson == True:
            self.ellipses_count = torch.poisson(torch.full((img_count,), ellipses_count).to(torch.float32), generator=generator).to(torch.int32)
        else:
            self.ellipses_count = (torch.full((img_count,), ellipses_count).to(torch.float32)).to(torch.int32)
        real_ellipses_count = self.ellipses_count.sum()
        self.ellipse_angle = torch.rand((real_ellipses_count,), generator=generator)*360.0
        self.ellipse_alpha = torch.rand((real_ellipses_count,), generator=generator)*0.9+0.1
        self.ellipse_width_aa  = torch.rand((real_ellipses_count,), generator=generator)*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_height_aa = torch.rand((real_ellipses_count,), generator=generator)*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_x_raw = torch.rand((real_ellipses_count,), generator=generator)
        self.ellipse_y_raw = torch.rand((real_ellipses_count,), generator=generator)

        r = torch.rand((real_ellipses_count,), generator=generator)*0.3
        alpha = torch.rand((real_ellipses_count,), generator=generator)*2.0*torch.pi
        self.ellipse_x_raw = torch.cos(alpha)*r+0.5
        self.ellipse_y_raw = torch.sin(alpha)*r+0.5
        if random_size:
            self.ellipse_width_aa  = torch.rand((real_ellipses_count,), generator=generator)*0.15*img_size+0.05*img_size#max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
            self.ellipse_height_aa = torch.rand((real_ellipses_count,), generator=generator)*0.15*img_size+0.05*img_size#max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        else:
            self.ellipse_width_aa  = torch.ones(real_ellipses_count)*ellipses_size #max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
            self.ellipse_height_aa = torch.ones(real_ellipses_count)*ellipses_size #max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min


        #self.ellipses_count = torch.ones((img_count,), dtype=torch.int32)
        #self.ellipse_width_aa  = torch.full((img_count,), img_size)#ellipses_size)
        #self.ellipse_height_aa = torch.full((img_count,), img_size)#ellipses_size)
        #self.ellipse_x_raw = torch.full((img_count,), 0.5)
        #self.ellipse_y_raw = torch.full((img_count,), 0.5)
        #self.ellipse_angle = torch.zeros((img_count,))
        #self.ellipse_alpha = torch.ones((img_count,))

        self.transform = transform

        #self.channel_selection_mode = "alpha"
        #if self[0][0].sum() == self.img_size*self.img_size:
        #    self.channel_selection_mode = "noalpha"
        self.channel_selection_mode = "noalpha"

        print(f'channel_selection_mode: {self.channel_selection_mode}')

    def __len__(self) -> int:
        return self.img_count

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, int]:
        fig = plt.figure(figsize=(self.img_size,self.img_size), dpi=1)
        ax = fig.add_axes([0.0,0.0,1.0,1.0])
        ellipse_func = lambda w, h, a, t: (w/2.0*cos(t)*cos(a)-h/2.0*sin(t)*sin(a), w/2.0*cos(t)*sin(a)+h/2.0*sin(t)*cos(a))
        prev_ellipses_count = self.ellipses_count[:idx].sum()
        for i in range(self.ellipses_count[idx]):
            e_idx = prev_ellipses_count+i
            args = (self.ellipse_width_aa[e_idx].item(), self.ellipse_height_aa[e_idx].item(), self.ellipse_angle[e_idx].item()/180.0*torch.pi)
            if self.ellipse_angle[idx] == 0.0:
                ellipse_width = self.ellipse_width_aa[e_idx]
                ellipse_height = self.ellipse_height_aa[e_idx]
            else:
                t = atan(-self.ellipse_height_aa[e_idx].item()*tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)/self.ellipse_width_aa[e_idx].item())
                ellipse_width = max(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])-min(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])
                t = atan(self.ellipse_height_aa[e_idx].item()/(tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)*self.ellipse_width_aa[e_idx].item()))
                ellipse_height = max(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])-min(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])
            ellipse_x = ellipse_width/2.0+self.ellipse_x_raw[e_idx].item()*(self.img_size-ellipse_width)
            ellipse_y = ellipse_height/2.0+self.ellipse_y_raw[e_idx].item()*(self.img_size-ellipse_height)
            ellipse = matplotlib.patches.Ellipse(xy=[ellipse_x, ellipse_y], width=self.ellipse_width_aa[e_idx].item(), height=self.ellipse_height_aa[e_idx].item(), angle=self.ellipse_angle[e_idx].item())
            ax.add_artist(ellipse)
            ellipse.set_clip_box(ax.bbox)
            ellipse.set_alpha(self.ellipse_alpha[e_idx].item())
            ellipse.set_facecolor("black")
            ellipse.set_edgecolor(None)
            ellipse.set_antialiased(False)
        ax.axis("off")
        ax.set_xlim(0.0, self.img_size)
        ax.set_ylim(0.0, self.img_size)
        fig.add_axes(ax)
        fig.canvas.draw()
        img = torch.from_numpy(np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8).copy())
        if self.channel_selection_mode == "noalpha":
            img = 1.0-torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[3:4]/255.0
        elif self.channel_selection_mode == "alpha":
            img = torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[0:1]/255.0
        else:
            raise NotImplementedError()
        if self.transform != None:
            img = self.transform(img)
        plt.close()
        return img, 0

#import random
#def seed_worker(worker_id):
#    try:
#        worker_seed = torch.cuda.initial_seed() % 2**32
#    except:
#        worker_seed = torch.initial_seed() % 2**32

#    np.random.seed(worker_seed)
#    random.seed(worker_seed)

class my_EllipsesDataModule(pl.LightningDataModule):
    def __init__(self, config: omegaconf.DictConfig) -> None:
        super().__init__()
        self.config = config.copy()
        self.training_generator = torch.Generator()
        #self.training_generator.manual_seed(1234567) #(torch.randint(0, 999_999_999_999_999, (1,)).item())
        self.validation_generator = torch.Generator()
        #self.validation_generator.manual_seed(1234568) #(torch.randint(0, 999_999_999_999_999, (1,)).item())
        self.test_generator = torch.Generator()
        #self.test_generator.manual_seed(1234569) #(torch.randint(0, 999_999_999_999_999, (1,)).item())

        v_compose = []
        if config.dataset.blurred:
            # v_compose.append(torchvision.transforms.Lambda(lambda x: torchvision.transforms.functional.gaussian_blur(x, 5, 2.5))) # old line (can be deleted)
            v_compose.append(torchvision.transforms.GaussianBlur(kernel_size=5, sigma=2.5))
            
        # v_compose.append(torchvision.transforms.Lambda(lambda x: torchvision.transforms.Resize(size=config.resized_shape)(x))) # old line (can be deleted)
        v_compose.append(torchvision.transforms.Resize(size=config.resized_shape)) # !!! IMPORTANT !!! comment this line if you want "original"

        if len(v_compose)==0:
            self.dataset_transform = None
        else:
            self.dataset_transform = torchvision.transforms.Compose(v_compose)

        #self.training_generator.manual_seed(1234567)
        #self.training_dataset = _my_EllipsesDataset((640 if self.config.training_batch_count == -1 else self.config.training_batch_count)*self.config.training_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.training_generator) # if self.config.dataset.blurred else None)
        #self.validation_generator.manual_seed(1234568)
        #self.validation_dataset = _my_EllipsesDataset((160 if self.config.validation_batch_count == -1 else self.config.validation_batch_count)*self.config.validation_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.validation_generator) # if self.config.dataset.blurred else None)
        self.test_generator.manual_seed(1234569)
        self.test_dataset = _my_EllipsesDataset((200 if self.config.test_batch_count == -1 else self.config.test_batch_count)*self.config.test_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.test_generator) # if self.config.dataset.blurred else None)
        

    #def train_dataloader(self) -> torch.utils.data.DataLoader:
    #    #self.training_generator.manual_seed(1234567)
    #    #training_dataset = _my_EllipsesDataset((640 if self.config.training_batch_count == -1 else self.config.training_batch_count)*self.config.training_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.training_generator) # if self.config.dataset.blurred else None)
    #    return torch.utils.data.DataLoader(self.training_dataset, drop_last=self.config.drop_last_training_batch, batch_size=self.config.training_batch_size, shuffle=self.config.shuffle_training_data, num_workers=self.config.num_workers, generator=self.training_generator)#, worker_init_fn=seed_worker)
    
    #def val_dataloader(self) -> torch.utils.data.DataLoader:
    #    #self.validation_generator.manual_seed(1234568)
    #    #validation_dataset = _my_EllipsesDataset((160 if self.config.validation_batch_count == -1 else self.config.validation_batch_count)*self.config.validation_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.validation_generator) # if self.config.dataset.blurred else None)
    #    return torch.utils.data.DataLoader(self.validation_dataset, drop_last=self.config.drop_last_validation_batch, batch_size=self.config.validation_batch_size, shuffle=self.config.shuffle_validation_data, num_workers=self.config.num_workers, generator=self.validation_generator)#, worker_init_fn=seed_worker)

    def test_dataloader(self) -> torch.utils.data.DataLoader:
        print("I'm in test_dataloader.")
        print(f"self.config.dataset.img_size is {self.config.dataset.img_size}")
        #self.test_generator.manual_seed(1234569)
        #test_dataset = _my_EllipsesDataset((200 if self.config.test_batch_count == -1 else self.config.test_batch_count)*self.config.test_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, self.dataset_transform, self.test_generator) # if self.config.dataset.blurred else None)
        #batch_size=self.config.test_batch_size
        return torch.utils.data.DataLoader(self.test_dataset, drop_last=self.config.drop_last_test_batch, batch_size=1, shuffle=self.config.shuffle_test_data, num_workers=self.config.num_workers, generator=self.test_generator)#, worker_init_fn=seed_worker)

In [26]:
my_datamodule_v0 = my_EllipsesDataModule(config_v3)

#test_generator = torch.Generator()
#test_generator.manual_seed(1234569) #(torch.randint(0, 999_999_999_999_999, (1,)).item())

my_test_datamodule_v0 = torch.utils.data.DataLoader(my_datamodule_v0.test_dataset, batch_size=config_v3.test_batch_size, shuffle=config_v3.shuffle_test_data) #, num_workers=config_v3.num_workers)

# torch.utils.data.DataLoader(my_datamodule_v0.test_dataset, drop_last=config_v3.drop_last_test_batch, batch_size=1, shuffle=config_v3.config.shuffle_test_data, num_workers=config_v3.num_workers, generator=test_generator)#, worker_init_fn=seed_worker)

channel_selection_mode: noalpha


In [22]:
config_v3.num_workers

4

In [18]:
config_v3.test_batch_size

32

In [31]:
cont = 0
for Y in my_test_datamodule_v0:
    cont += 1
    break
    #all_images.append(X[0])

print(cont)

1


In [28]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

In [29]:
plt.imshow(Y[0][1].view(Y[0][1].shape[-2:]))
plt.show()

In [26]:
my_datamodule_v0.test_dataset[0][0].shape

torch.Size([1, 128, 128])

In [28]:
plt.imshow(my_datamodule_v0.test_dataset[0][0].view(my_datamodule_v0.test_dataset[0][0].shape[-2:]))
plt.show()

In [30]:
aus_image = Y[0][1].view(Y[0][1].shape[-2:]).clone() # my_datamodule_v0.test_dataset[0][0].view(my_datamodule_v0.test_dataset[0][0].shape[-2:]).clone()

In [32]:
fig, axs = plt.subplots(2)
fig.suptitle('Test Ellipses')
axs[0].imshow(Y[0][1].view(Y[0][1].shape[-2:]))
#axs[0].imshow(my_datamodule_v0.test_dataset[0][0].view(my_datamodule_v0.test_dataset[0][0].shape[-2:]))
axs[1].imshow(aus_image)
plt.show()

In [89]:
[dataset_transform(X[0]), torch.arange(0,32,1)]

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [82]:
range_idx = torch.arange(0,32,1)

In [87]:
range_idx

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [18]:
import torchvision

In [19]:
v_compose = []
v_compose.append(torchvision.transforms.Resize(size=256))#config.resized_shape))

if len(v_compose)==0:
    dataset_transform = None
else:
    dataset_transform = torchvision.transforms.Compose(v_compose)

In [51]:
dataset_transform

Compose(
    Resize(size=64, interpolation=bilinear, max_size=None, antialias=None)
)

In [52]:
my_img.shape

torch.Size([1, 256, 256])

In [53]:
my_resized_img = dataset_transform(my_img)
my_resized_img.shape

torch.Size([1, 64, 64])

In [54]:
fig, axs = plt.subplots(2)
fig.suptitle('Test Ellipses')
axs[0].imshow(my_img.view(my_img.shape[-2:]))
axs[1].imshow(my_resized_img.view(my_resized_img.shape[-2:]))
plt.show()

In [17]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Test Ellipses')
axs[0,0].imshow(X[0][61].view(X[0][0].shape[-2:]))
axs[0,1].imshow(X[0][68].view(X[0][0].shape[-2:]))
axs[1,0].imshow(X[0][5].view(X[0][0].shape[-2:]))
axs[1,1].imshow(X[0][23].view(X[0][0].shape[-2:]))
plt.show()

In [18]:
print(config_v2)

{'model': {'name': 'learned', 'initialization': 'zeros'}, 'dataset': {'name': 'ellipses', 'img_size': 64, 'ellipse_count': 1, 'ellipse_size': 16, 'ellipse_size_min': 16, 'blurred': False}, 'out_dir': './LearnedRadonFilters', 'deterministic': True, 'seed': 1234567, 'device': 'cuda', 'num_workers': 4, 'checkpoint': None, 'validation_split_percent': 20, 'training_batch_size': 32, 'shuffle_training_data': True, 'drop_last_training_batch': False, 'validation_batch_size': '${training_batch_size}', 'shuffle_validation_data': False, 'drop_last_validation_batch': False, 'test_batch_size': 1000, 'shuffle_test_data': False, 'drop_last_test_batch': True, 'sino_angles': None, 'sino_positions': None, 'noise_level': 0.004, 'optimizer_lr': 0.05, 'epochs': 1, 'training_batch_count': -1, 'validation_batch_count': -1, 'test_batch_count': 1}


In [23]:
import torchvision.transforms as T
import torchvision

In [ ]:
T.Resize(size=64)(img)

In [22]:
config_v2.dataset.blurred

False

In [24]:
v_compose = []

# v_compose.append(torchvision.transforms.Lambda(lambda x: torchvision.transforms.functional.gaussian_blur(x, 5, 2.5)))

v_compose.append(torchvision.transforms.Lambda(lambda x: torchvision.transforms.Resize(size=64)(x)))

test_transform = torchvision.transforms.Compose(v_compose)

In [26]:
config = config_v2

In [28]:
config.dataset.blurred = True

In [29]:
test_transform

Compose(
    Lambda()
)

In [31]:
#pyright: reportGeneralTypeIssues=false

from math import atan, cos, sin, tan
import typing

import omegaconf
import pytorch_lightning as pl
import torch
import torch.utils.data
import torchvision.transforms
import torchvision.transforms.functional
import matplotlib
matplotlib.use("agg")
import matplotlib.patches
import matplotlib.pyplot as plt
import numpy as np
matplotlib.use('TkAgg')

import sys
check_python_version = (sys.version_info[0] >= 3) and (sys.version_info[1] >= 10)

class _EllipsesDataset(torch.utils.data.Dataset):
    def __init__(self, img_count: int, img_size: int, ellipses_count: int, ellipses_size: float, ellipses_size_min: float=1, transform: typing.Callable[[torch.Tensor],torch.Tensor]|None if check_python_version else typing.Union[typing.Callable[[torch.Tensor],torch.Tensor] , None] = None, number_ellipses_poisson=False, random_size=False):
        self.img_count = img_count
        self.img_size = img_size
        if number_ellipses_poisson == True:
            self.ellipses_count = torch.poisson(torch.full((img_count,), ellipses_count).to(torch.float32)).to(torch.int32)
        else:
            self.ellipses_count = (torch.full((img_count,), ellipses_count).to(torch.float32)).to(torch.int32)
        real_ellipses_count = self.ellipses_count.sum()
        self.ellipse_angle = torch.rand((real_ellipses_count,))*360.0
        self.ellipse_alpha = torch.rand((real_ellipses_count,))*0.9+0.1
        self.ellipse_width_aa  = torch.rand((real_ellipses_count,))*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_height_aa = torch.rand((real_ellipses_count,))*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_x_raw = torch.rand((real_ellipses_count,))
        self.ellipse_y_raw = torch.rand((real_ellipses_count,))

        r = torch.rand((real_ellipses_count,))*0.3
        alpha = torch.rand((real_ellipses_count,))*2.0*torch.pi
        self.ellipse_x_raw = torch.cos(alpha)*r+0.5
        self.ellipse_y_raw = torch.sin(alpha)*r+0.5
        if random_size:
            self.ellipse_width_aa  = torch.rand((real_ellipses_count,))*0.15*img_size+0.05*img_size#max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
            self.ellipse_height_aa = torch.rand((real_ellipses_count,))*0.15*img_size+0.05*img_size#max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        else:
            self.ellipse_width_aa  = torch.ones(real_ellipses_count)*ellipses_size #max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
            self.ellipse_height_aa = torch.ones(real_ellipses_count)*ellipses_size #max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min


        #self.ellipses_count = torch.ones((img_count,), dtype=torch.int32)
        #self.ellipse_width_aa  = torch.full((img_count,), img_size)#ellipses_size)
        #self.ellipse_height_aa = torch.full((img_count,), img_size)#ellipses_size)
        #self.ellipse_x_raw = torch.full((img_count,), 0.5)
        #self.ellipse_y_raw = torch.full((img_count,), 0.5)
        #self.ellipse_angle = torch.zeros((img_count,))
        #self.ellipse_alpha = torch.ones((img_count,))

        self.transform = transform

        self.channel_selection_mode = "alpha"
        if self[0][0].sum() == self.img_size*self.img_size:
            self.channel_selection_mode = "noalpha"

    def __len__(self) -> int:
        return self.img_count

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, int]:
        fig = plt.figure(figsize=(self.img_size,self.img_size), dpi=1)
        ax = fig.add_axes([0.0,0.0,1.0,1.0])
        ellipse_func = lambda w, h, a, t: (w/2.0*cos(t)*cos(a)-h/2.0*sin(t)*sin(a), w/2.0*cos(t)*sin(a)+h/2.0*sin(t)*cos(a))
        prev_ellipses_count = self.ellipses_count[:idx].sum()
        for i in range(self.ellipses_count[idx]):
            e_idx = prev_ellipses_count+i
            args = (self.ellipse_width_aa[e_idx].item(), self.ellipse_height_aa[e_idx].item(), self.ellipse_angle[e_idx].item()/180.0*torch.pi)
            if self.ellipse_angle[idx] == 0.0:
                ellipse_width = self.ellipse_width_aa[e_idx]
                ellipse_height = self.ellipse_height_aa[e_idx]
            else:
                t = atan(-self.ellipse_height_aa[e_idx].item()*tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)/self.ellipse_width_aa[e_idx].item())
                ellipse_width = max(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])-min(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])
                t = atan(self.ellipse_height_aa[e_idx].item()/(tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)*self.ellipse_width_aa[e_idx].item()))
                ellipse_height = max(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])-min(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])
            ellipse_x = ellipse_width/2.0+self.ellipse_x_raw[e_idx].item()*(self.img_size-ellipse_width)
            ellipse_y = ellipse_height/2.0+self.ellipse_y_raw[e_idx].item()*(self.img_size-ellipse_height)
            ellipse = matplotlib.patches.Ellipse(xy=[ellipse_x, ellipse_y], width=self.ellipse_width_aa[e_idx].item(), height=self.ellipse_height_aa[e_idx].item(), angle=self.ellipse_angle[e_idx].item())
            ax.add_artist(ellipse)
            ellipse.set_clip_box(ax.bbox)
            ellipse.set_alpha(self.ellipse_alpha[e_idx].item())
            ellipse.set_facecolor("black")
            ellipse.set_edgecolor(None)
            ellipse.set_antialiased(False)
        ax.axis("off")
        ax.set_xlim(0.0, self.img_size)
        ax.set_ylim(0.0, self.img_size)
        fig.add_axes(ax)
        fig.canvas.draw()
        img = torch.from_numpy(np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8).copy())
        if self.channel_selection_mode == "noalpha":
            img = 1.0-torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[3:4]/255.0
        elif self.channel_selection_mode == "alpha":
            img = torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[0:1]/255.0
        else:
            raise NotImplementedError()
        plt.close()
        if self.transform != None:
            img = self.transform(img)
        return img, 0

In [32]:
test_dataset = _EllipsesDataset((200 if config.test_batch_count == -1 else config.test_batch_count)*config.test_batch_size, config.dataset.img_size, config.dataset.ellipse_count, config.dataset.ellipse_size, config.dataset.ellipse_size_min, test_transform if config.dataset.blurred else None)

In [34]:
len(test_dataset)

1000

In [36]:
config.test_batch_size

1000

In [40]:
test_dataset[4][0].shape

torch.Size([1, 64, 64])

In [45]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Test Ellipses')
axs[0,0].imshow(test_dataset[4][0].view(test_dataset[4][0].shape[-2:]))
axs[0,1].imshow(test_dataset[68][0].view(test_dataset[4][0].shape[-2:]))
axs[1,0].imshow(test_dataset[6][0].view(test_dataset[4][0].shape[-2:]))
axs[1,1].imshow(test_dataset[23][0].view(test_dataset[4][0].shape[-2:]))
plt.show()

In [55]:
# genero tutte le immagini che mi servono
# Train

# Test

# Validation


In [ ]:
# Le metto ognuna in in utils.Dataset